Jupyter Notebook For ReadMyMind, A CS 125 @ Illinois MP7 Project by Isaac Park and Mihir Pandya.

In [5]:
import tweepy
import pandas as pd
from textblob import TextBlob
from datetime import datetime

username = 'realDonaldTrump' # working example: Donald Trump

auth = tweepy.OAuthHandler('1X5fCqPl7yVvYxQjQJwkvavFD', 'NXbTDPP3HxlXOL5dWdCegEP09odLAkxUWlyRvZqXxtAtdX597G')
auth.set_access_token('925495606931546112-mn3Hda41LsZhbYAKJtddL7TulRKucuj', 'lvCFqSLv5YvOGzCINH6JZ5cBI1CEkPKrRioBn5Iuec3Tt')
api = tweepy.API(auth)
    
tweets_df = pd.DataFrame({
    'Timestamp': (),
    'Likes': (),
    'Retweets': (),
    'Text': (),
    'Sentences': (),
    'Sentiment_Total': (),
    'Keywords': ()
})

tweets_df = tweets_df[['Timestamp', 'Likes', 'Retweets', 'Text', 'Sentences', 'Sentiment_Total', 'Keywords']]
    
recent_tweets = api.user_timeline(screen_name = username, count=100, tweet_mode="extended") # analyzing 100 tweets
for status in recent_tweets:
    test = status.full_text
    if test[:2] != 'RT': # removing retweets made by the user
        status_data = pd.Series([status.created_at, status.favorite_count, status.retweet_count, status.full_text], 
                                index=['Timestamp', 'Likes', 'Retweets', 'Text'])
    tweets_df = tweets_df.append(status_data, ignore_index = True)
    
tweets_df = tweets_df.drop_duplicates(subset='Text') # just in case, remove any duplicate tweets
tweets_df = tweets_df.astype('object')

for i in range(len(tweets_df)):
    content = tweets_df.iloc[i]['Text']
    if 'http' in content:
        j = content.index('http')
        content = content[:j] # cleaning text of the tweet by removing the link at the end and newline characters
    content = content.replace('\n', '')
    tweets_df.iloc[i]['Text'] = content
    
    blob = TextBlob(content)
    tweets_df.iloc[i]['Sentiment_Total'] = blob.sentiment.subjectivity
    sentiments = {}
    
    for sent in blob.sentences: # generating sentiment polarity values for each sentence in the tweet
        sentiments[str(sent)] = sent.sentiment.subjectivity
        
    tweets_df.iloc[i]['Sentences'] = sentiments # insert dictionary of sentence: sentiment value into dataframe
    
    tweets_df.iloc[i]['Timestamp'] = tweets_df.iloc[i]['Timestamp'].to_pydatetime() # convert pandas.tslib.Timestamp object to datetime
    
    # Keyword extraction goes here
    # append list of keywords

tweets_df

,Timestamp,Likes,Retweets,Text,Sentences,Sentiment_Total,Keywords
0,2017-12-08 15:18:27,41087,10213,Fines and penalties against Wells Fargo Bank f...,{'Fines and penalties against Wells Fargo Bank...,0.666667,NaN
1,2017-12-08 15:06:55,32672,8876,LAST thing the Make America Great Again Agenda...,{'LAST thing the Make America Great Again Agen...,0.416667,NaN
2,2017-12-08 13:11:43,46827,10010,"Big crowd expected today in Pensacola, Florida...","{'Big crowd expected today in Pensacola, Flori...",0.41,NaN
3,2017-12-08 05:41:08,99397,35275,I fulfilled my campaign promise - others didn’t!,{'I fulfilled my campaign promise - others did...,0,NaN
4,2017-12-08 05:36:09,32860,7238,"Tonight, @FLOTUS Melania and I were thrilled t...","{'Tonight, @FLOTUS Melania and I were thrilled...",0.68,NaN
5,2017-12-07 21:10:28,52990,12093,"Across the battlefields, oceans, and harrowing...","{'Across the battlefields, oceans, and harrowi...",0.75,NaN
6,2017-12-07 20:52:49,52847,11702,"Today, as we Remember Pearl Harbor, it was an ...","{'Today, as we Remember Pearl Harbor, it was a...",0.45,NaN
7,2017-12-07 20:04:20,46969,9699,"Today, the U.S. flag flies at half-staff at th...","{'Today, the U.S. flag flies at half-staff at ...",0,NaN
8,2017-12-07 16:16:19,74067,21120,"Today, our entire nation pauses to REMEMBER PE...","{'Today, our entire nation pauses to REMEMBER ...",0.8125,NaN
9,2017-12-07 15:04:54,99508,22431,National Pearl Harbor Remembrance Day - “A day...,{'National Pearl Harbor Remembrance Day - “A d...,0.5,NaN


Ideas for popular keyword extraction:

1. Tokenize each word with part of speech tag; keep only proper nouns and nouns.
2. Record frequency of each word; only keep words that occur above a certain number of times (frequency threshold). These will be our "keywords".
3. Put the list of keywords for each tweet into the 'Keywords' column of the dataframe.

Ideas for graphing the keywords/frequency/likes/retweets relationships:

1. y-axis: frequencies of keywords, x-axis: time; line graph with each line representing the popularity of one keyword over time.

2. y-axis: likes/retweet count, x-axis: frequencies of keywords; scatter plot with each dot representing a keyword.

any more possible visualization suggestions?

